<h1 style="background: orange; color: #5E7AFF; text-align: center; line-height: 200%; letter-spacing: 1px; font-weight: bold;">Permutations with Lehmer</h1>
<h4>Peter Luschny, July 2022</h4>

##### Outline:

* Permutations

	=> select by property of associated Lehmer code

* Partitions

	=> sort by [A334439](https://oeis.org/A334439)

* Partition Triangle (0 <= k <= [A000041](https://oeis.org/A000041)(n))

	=> sum by largest part

* Reduced Triangle (0 <= k <= n)

	=> row sum

* Sequence



##### Concerned with:

|Partition <br> triangle|Reduced <br> triangle|Sequence|Lehmer <br> constraint|
| :---:   | :---:   | :---:   | :---:  |
| [A355777](https://oeis.org/A355777) | [A173018](https://oeis.org/A173018) | [A000142](https://oeis.org/A000142) | none |
| [A355776](https://oeis.org/A355776) | [A356116](https://oeis.org/A356116) | [A056986](https://oeis.org/A56986)  | not monotonic |
| [A356113](https://oeis.org/A356113) | [A174159](https://oeis.org/A174159) | [A356118](https://oeis.org/A356118) | -- |
| [A134264](https://oeis.org/A134264) | [A001263](https://oeis.org/A001263) | [A000108](https://oeis.org/A000108) | weakly <br> decreasing |

##### Comment:

<p style="color:brown;font-size:large">By a partition triangle, we understand an irregular triangle where each row corresponds to a mapping of Partitions(n) -> ZZ. We assume a fixed order of the partitions given. Here we will use the ordering defined by Gus Wiseman in A334439, which is the Abramowitz-Stegun ordering, except that the finer order is reverse-lexicographic instead of lexicographic. Examples are A355776, A355777, and A134264. 'Reducing' then means summing the values corresponding to the partitions of n with length k. The 'reduced partition triangle' then is a regular triangle with T(n, k) with 1 <= k <= n.</p>

<p style="color:brown;font-size:large">Conversely, for instance, A355776, the statistic of permutations whose Lehmer code is nonmonotonic, can be seen as a refinement of A356116, which in turn is a refinement of the sequence A056986, the number of permutations on [n] containing any given pattern alpha in the symmetric group S_3.</p>

In [1]:
import collections
from functools import cache
from typing import Callable
from sage.all import ZZ
from sage.combinat.permutation import Permutation, Permutations
from sage.combinat.partition import Partition, Partitions, number_of_partitions

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Some utility functions used later as constrains.
</h3>

In [2]:
def weakly_increasing(L: list[int]) -> bool:
    return all(x <= y for x, y in zip(L, L[1:]))


def weakly_decreasing(L: list[int]) -> bool:
    return all(x >= y for x, y in zip(L, L[1:]))


def not_monotonic(L: list[int]) -> bool:
    return not weakly_increasing(L) and not weakly_decreasing(L)


def void(L) -> bool:
    return True

<p style="color:brown;font-size:large">Utility function. Reduce a partition triangle to a 'normal' triangle.</p> 

In [3]:
@cache
def Pn(n: int, k: int) -> int:
    if k == 0: return 0
    if n == 0 or k == 1: return 1
    return Pn(n, k - 1) + Pn(n - k, k) if k <= n else Pn(n, k - 1)


def reduce_parts(fun: Callable, n: int) -> list[int]:
    funn: list[int] = fun(n)
    return [sum(funn[Pn(n, k):Pn(n, k + 1)]) for k in range(n)]


def reduce_partition_triangle(fun: Callable, n: int) -> list[list[int]]:
    return [reduce_parts(fun, k) for k in range(1, n)]

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Permutation statistics
</h3>

<p style="color:brown;font-size:large">The main function. Maps permutations to partitions subject to constraints on their Lehmer code.</p>

<p style="color:brown;font-size:large">The order of the integer partitions is the Abramowitz-Stegun ordering except that the finer order is reverse-lexicographic instead of lexicographic, as defined by Gus Wiseman in A334439.</p>

In [4]:
def perm_stats(n: int, constraint: Callable):
    res = collections.defaultdict(int)
    for p in Permutations(n):
        l: list[int] = p.to_lehmer_code()
        if constraint(l):
            c: list[int] = [l.count(i) for i in range(len(p)) if i in l]
            res[Partition(reversed(sorted(c)))] += 1
    return sorted(res.items(), key=lambda x: len(x[0]))

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Example of the various mappings involved, the permutations of {1, 2, 3, 4}.
</h3>
<p style="color:brown;font-size:large">The 'type' indicates whether the Lehmer code is weakly increasing, weakly decreasing or not monotonic.</p>


|Permutation|Lehmer Code|Type|Partition|
| :---:   | :---:   | :---:   | :--- |
|1234| [0000]| [T,T,F]| [4]|       
|1243| [0010]| [F,F,T]| [3, 1]|    
|1324| [0100]| [F,F,T]| [3, 1]|    
|1423| [0200]| [F,F,T]| [3, 1]|    
|2134| [1000]| [F,T,F]| [3, 1]|    
|3124| [2000]| [F,T,F]| [3, 1]|    
|4123| [3000]| [F,T,F]| [3, 1]|    
|2341| [1110]| [F,T,F]| [3, 1]|    
|1342| [0110]| [F,F,T]| [2, 2]|    
|2143| [1010]| [F,F,T]| [2, 2]|    
|2314| [1100]| [F,T,F]| [2, 2]|    
|3412| [2200]| [F,T,F]| [2, 2]|    
|1432| [0210]| [F,F,T]| [2, 1, 1]| 
|2413| [1200]| [F,F,T]| [2, 1, 1]| 
|2431| [1210]| [F,F,T]| [2, 1, 1]| 
|3142| [2010]| [F,F,T]| [2, 1, 1]| 
|3214| [2100]| [F,T,F]| [2, 1, 1]| 
|3241| [2110]| [F,T,F]| [2, 1, 1]| 
|3421| [2210]| [F,T,F]| [2, 1, 1]| 
|4132| [3010]| [F,F,T]| [2, 1, 1]| 
|4213| [3100]| [F,T,F]| [2, 1, 1]| 
|4231| [3110]| [F,T,F]| [2, 1, 1]| 
|4312| [3200]| [F,T,F]| [2, 1, 1]| 
|4321| [3210]| [F,T,F]| [1, 1, 1, 1]| 

<p style="color:brown;font-size:large">This table can be nicely summarized in this mapping:</p>

|Partition|Counts|
| :--: | :--: |
|[4]| 1|
|[3, 1]| 7|
|[2, 2]| 4|
|[2, 1, 1]| 11|
|[1, 1, 1, 1] |1|

<p style="color:brown;font-size:large">The counts are row 4 of A355777.</p>

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A355777
</h3>


<p style="color:brown;font-size:large">A355777 -- A statistic of permutations given by their Lehmer code refining Euler's triangle A173018.</p>

In [5]:
@cache
def A355777_row(n: int) -> list[int]:
    return [v[1] for v in perm_stats(n, void)]


def A355777(n: int, k: int) -> int:
    return A355777_row(n)[k]

In [6]:
for n in range(8):
    print(A355777_row(n))

# for n in range(8):
#    print([A355777(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[1, 1]
[1, 4, 1]
[1, 7, 4, 11, 1]
[1, 11, 15, 32, 34, 26, 1]
[1, 16, 26, 15, 76, 192, 34, 122, 180, 57, 1]
[1, 22, 42, 56, 156, 474, 267, 294, 426, 1494, 496, 423, 768, 120, 1]


<p style="color:brown;font-size:large">Euler's A173018 -- Reduced partition triangle A355777.</p> 

In [7]:
T = reduce_partition_triangle(A355777_row, 9)
for row in T: print(row)

[1]
[1, 1]
[1, 4, 1]
[1, 11, 11, 1]
[1, 26, 66, 26, 1]
[1, 57, 302, 302, 57, 1]
[1, 120, 1191, 2416, 1191, 120, 1]
[1, 247, 4293, 15619, 15619, 4293, 247, 1]


<p style="color:brown;font-size:large">A000142 -- Factorial numbers, row sums of A173018.</p>

In [8]:
[sum(r) for r in reduce_partition_triangle(A355777_row, 9)]

[1, 2, 6, 24, 120, 720, 5040, 40320]

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A355776
</h3>



<p style="color:brown;font-size:large">A355776 -- A statistic of permutations whose Lehmer code is nonmonotonic, refining triangle A356116.</p>


In [9]:
@cache
def A355776_row(n: int) -> list[int]:
    if n < 2:
        return [0]
    S = perm_stats(n, not_monotonic)
    return [0] + [s[1] for s in S] + [0]


def A355776(n: int, k: int) -> int:
    return A355776_row(n)[k] if n > 0 else 0

In [10]:
for n in range(8):
    print(A355776_row(n))

#for n in range(8):
#    print([A355776(n, k) for k in range(number_of_partitions(n))])

[0]
[0]
[0, 0]
[0, 1, 0]
[0, 3, 2, 5, 0]
[0, 6, 10, 22, 24, 16, 0]
[0, 10, 20, 12, 61, 162, 29, 102, 150, 42, 0]


[0, 15, 35, 49, 135, 432, 246, 273, 391, 1389, 461, 388, 698, 99, 0]


<p style="color:brown;font-size:large">A356116 -- Reduced partition triangle A355776.</p> 

In [11]:
T = reduce_partition_triangle(A355776_row, 9)
for row in T: print(row)

[0]
[0, 0]
[0, 1, 0]
[0, 5, 5, 0]
[0, 16, 46, 16, 0]
[0, 42, 252, 252, 42, 0]
[0, 99, 1086, 2241, 1086, 99, 0]
[0, 219, 4097, 15129, 15129, 4097, 219, 0]


<p style="color:brown;font-size:large">A056986 -- Number of permutations on {1,...,n} containing any given pattern alpha in the symmetric group S_3.</p>

In [12]:
[sum(r) for r in reduce_partition_triangle(A355776_row, 9)]

[0, 0, 1, 10, 78, 588, 4611, 38890]

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A356113
</h3>


<p style="color:brown;font-size:large">A356113 -- Refining A174159, the Euler minus Narayana/Catalan triangle.<br> 
    T(n, k) = A355777(n, k) + A355776(n, k).</p>

In [13]:
@cache
def A356113(n: int, k: int) -> int:
    return A355777(n, k) + A355776(n, k)


def A356113_row(n: int) -> list[int]:
    return [A356113(n, k) for k in range(number_of_partitions(n))]

In [14]:
for n in range(8):
    print(A356113_row(n))

#for n in range(8):
#    print([A356113(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[1, 1]
[1, 5, 1]
[1, 10, 6, 16, 1]
[1, 17, 25, 54, 58, 42, 1]
[1, 26, 46, 27, 137, 354, 63, 224, 330, 99, 1]
[1, 37, 77, 105, 291, 906, 513, 567, 817, 2883, 957, 811, 1466, 219, 1]


<p style="color:brown;font-size:large">A174159 -- Reduced partition triangle A356113.</p>

In [15]:
T = reduce_partition_triangle(A356113_row, 8)
for row in T: print(row)

[1]
[1, 1]
[1, 5, 1]
[1, 16, 16, 1]
[1, 42, 112, 42, 1]
[1, 99, 554, 554, 99, 1]
[1, 219, 2277, 4657, 2277, 219, 1]


<p style="color:brown;font-size:large">A356118 -- Row sums of A174159.</p>

In [16]:
[sum(r) for r in reduce_partition_triangle(A356113_row, 9)]

[1, 2, 7, 34, 198, 1308, 9651, 79210]

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
A134264
</h3>


<p style="color:brown;font-size:large">A134264 -- Noncrossing partitions. <br>
Refining A001263, the triangle of Narayana numbers.<br>
    T(n, k) = A355777(n, k) - A355776(n, k).</p>

In [17]:
@cache
def A134264(n: int, k: int) -> int:
    return A355777(n, k) - A355776(n, k)


def A134264_row(n: int) -> list[int]:
    return [A134264(n, k) for k in range(number_of_partitions(n))]

In [18]:
for n in range(8):
    print(A134264_row(n))

#for n in range(7):
#    print([A134264(n, k) for k in range(number_of_partitions(n))])

[1]
[1]
[1, 1]
[1, 3, 1]
[1, 4, 2, 6, 1]
[1, 5, 5, 10, 10, 10, 1]
[1, 6, 6, 3, 15, 30, 5, 20, 30, 15, 1]
[1, 7, 7, 7, 21, 42, 21, 21, 35, 105, 35, 35, 70, 21, 1]


<p style="color:brown;font-size:large">A001263 -- Triangle of Narayana numbers.</p>

In [19]:
T = reduce_partition_triangle(A134264_row, 8)
for row in T: print(row)

[1]
[1, 1]
[1, 3, 1]
[1, 6, 6, 1]
[1, 10, 20, 10, 1]
[1, 15, 50, 50, 15, 1]
[1, 21, 105, 175, 105, 21, 1]


<p style="color:brown;font-size:large">A000108 -- Catalan numbers.</p>

In [20]:
[sum(r) for r in reduce_partition_triangle(A134264_row, 9)]

[1, 2, 5, 14, 42, 132, 429, 1430]